In [ ]:
import pandas as pd
import nomenclature
import pyam

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - Perceptions",
    scenario="Observed Data",
)

In [ ]:
df_list = list()

# Indicator 23

In [ ]:
data = (
    pd.read_csv("source/6_Perceptions/Ch.6_Fig3a_Indicator23_twitter_counts_by_cdr_method.csv")
    .set_index("yr")
    .stack()
) / 1000

In [ ]:
data.index.names = ["year", "variable"]

In [ ]:
df_23 = pyam.IamDataFrame(
    data,
    **iamc_args,
    unit="thousand",
    region="World",
)

In [ ]:
variable_prefix = "Social Media|Number of Tweets"

df_23.rename(
    variable={
        "Afforestation - Reforestation": "Afforestation and Reforestation",
        "BECCS": "Bioenergy with CCS",
        "Direct Air Capture": "Direct Air Capture with CCS",
        "Ecosystem Restoration": "Peatland Restoration",
        "Blue Carbon": "Coastal Wetland Restoration",
        "GGR (general)": "General CDR",
        "Ocean Alkalinization": "Ocean Alkalinity Enhancement",
    },
    inplace=True,
)
df_23.rename(
    variable=dict([(i, f"{variable_prefix}|{i}") for i in df_23.variable]),
    inplace=True,
)
df_23.aggregate(variable_prefix, append=True)

df_list.append(df_23)

In [ ]:
dsd.validate(df_23)

In [ ]:
df_23.filter(variable=f"{variable_prefix}|*").plot.bar(stacked=True)

# Indicator 26

In [ ]:
df_26 = pyam.IamDataFrame(
    "source/6_Perceptions/figure-data-cdr-media_v2.xlsx",
    sheet_name="figure_6.4",
    **iamc_args,
    variable="var",
    unit="",
    region="World",
)

In [ ]:
variable_prefix = "Newspapers|All Publications|Number of Articles"

df_26.rename(
    variable={
        "Afforestation/Reforestation": "Afforestation and Reforestation",
        "BECCS": "Bioenergy with CCS",
        "Coastal wetland restoration": "Coastal Wetland Restoration",
        "DAC(CS)": "Direct Air Capture with CCS",
        "Enhanced rock weathering": "Enhanced Weathering",
        "Ocean fertilisation": "Ocean Fertilization",
        "Peatland restoration": "Peatland Restoration",
        "Soil carbon sequestration": "Soil Carbon Sequestration",
    },
    inplace=True,
)
df_26.rename(
    variable=dict([(i, f"{variable_prefix}|{i}") for i in df_26.variable]),
    inplace=True,
)
df_26.aggregate(variable_prefix, append=True)

df_list.append(df_26)

In [ ]:
dsd.validate(df_26)

In [ ]:
df_26.filter(variable=f"{variable_prefix}|*").plot.bar(stacked=True)

In [ ]:
data = pd.read_excel("source/6_Perceptions/figure-data-cdr-media_v2.xlsx", sheet_name="figure_6.5").drop(columns="order")
data["region"] = data["label"].apply(lambda x: x.split("(")[1].split(")")[0])
data["label"] = data["label"].apply(lambda x: x.split(" (")[0])

In [ ]:
data["category"] = "Newspapers"
data["indicator"] = "Number of Articles"

In [ ]:
df_26_individual = pyam.IamDataFrame(
    data.drop(columns=["total", "source", "source_country"]),
    **iamc_args,
    variable=["category", "label", "indicator", "var"],
    unit="",
)

In [ ]:
newspapers = data["label"].unique()
list(newspapers)

In [ ]:
df_26_individual.rename(
    variable=dict(
        [
            (
                i, 
                i.replace("Afforestation/Reforestation", "Afforestation and Reforestation")
                .replace("BECCS", "Bioenergy with CCS")
                .replace("Coastal wetland restoration", "Coastal Wetland Restoration")
                .replace("DAC(CS)", "Direct Air Capture with CCS")
                .replace("Enhanced rock weathering", "Enhanced Weathering")
                .replace("Soil carbon sequestration", "Soil Carbon Sequestration")
                .replace("Ocean fertilisation", "Ocean Fertilization")
                .replace("Peatland restoration", "Peatland Restoration")

        ) for i in df_26_individual.variable]
    ),
    inplace=True,
)

In [ ]:
for newspaper in newspapers:
    df_26_individual.aggregate(f"Newspapers|{newspaper}|Number of Articles", append=True)

In [ ]:
dsd.validate(df_26_individual)

In [ ]:
df_list.append(df_26_individual)

# Merge data

In [ ]:
df = pyam.concat(df_list)

In [ ]:
dsd.validate(df)

In [ ]:
df.set_meta("Chapter 6", "Chapter")

In [ ]:
#df.to_ixmp4("socdr-dev")

In [ ]:
import ixmp4

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
run = platform.runs.get(**iamc_args)

In [ ]:
run.iamc.remove(run.iamc.tabulate())

In [ ]:
run.iamc.add(df.data)

In [ ]:
df.to_excel("socdr-2024_chapter-6_v1.xlsx")